# Projeto Mulheres Vítimas Versão 3
## Classificação de municípios quanto ao risco de violência contra mulher


**João L. S. Júnior**\
Universidade Federal do Pará - UFPA \
Laboratório de Pesquisa Operacional - LPO\
Instituto Federal de Educação, Ciência e Tecnologia do Pará - IFPA\
Copyright(c) 2023.


---

Comunicação com API IBGE

In [5]:
from generators.IBGEDataFetcher import IBGEDataFetcher


# Exemplo de uso:
state = 'pa'  # Exemplo: Pará
fetcher = IBGEDataFetcher(state)
result = fetcher.fetch_municipios_info()
result

,id,nome,IAP,IDH,PIB,GINI
0,150010,Abaetetuba,0.45,0.628,9458.72,0.40
1,150013,Abel Figueiredo,0.47,0.622,11420.76,0.37
2,150020,Acará,0.12,0.506,13582.97,0.36
3,150030,Afuá,0.12,0.489,9405.70,0.39
4,150034,Água Azul do Norte,0.27,0.564,15819.85,0.37
...,...,...,...,...,...,...
139,150815,Uruará,0.15,0.589,13401.03,0.36
140,150820,Vigia,0.32,0.617,7216.02,0.37
141,150830,Viseu,0.47,0.515,7952.36,0.38
142,150835,Vitória do Xingu,0.00,0.596,270414.24,0.39
